In [1]:
from DefSD import *


In [171]:
#   a table resultListDT relates the name of the objects and the resulting temoerature and density
#   for the four couples of species

resultListDT=Table.read('resultListDT.tbl', format='ascii')
ion = [' [OIII]', ' [SII]', ' [NII]', ' [OII]', ' [NII]', ' [SII]', ' [OIII]', ' [OII]']
for i in range(1,5):
    colT= Column(name='Temp_'+str(i)+ion[2*(i-1)], length=len(resultListDT),unit=u.K)
    colD= Column(name='Den_'+str(i)+ion[2*(i-1)+1], length=len(resultListDT),unit=1 / (u.cm**3))
    resultListDT.add_columns([colT,colD])
resultListDT

#   Obtain the lists Spectra, Headers, spectrum variance of the intensity according with a list of galaxy names
#   and a Table Tabla of models for each line for all the spectra

Headers = []
Spectra = []
SpectraCont = []
Lambda = []
Modelos=[]
Tablas=[]
HDUs=[]
TypeFITS=[]
SpectraIvar = []
LINEProfile = []
paramCont=0.1
typeFITS =0
for x in range(len(resultListDT['ObjID'])):
    name = resultListDT['ObjID'][x]
    z=resultListDT['z'][x]
    isFits = resultListDT['isFits'][x]
    if isFits:
        hdulist = fits.open('Exam_fits/'+name+'.fits')
        head= hdulist[0].header
        typeFITS=0
        if head['NAXIS'] == 0:
            total= hdulist[1].data
        elif head['NAXIS'] == 1:
            typeFITS=1
        if typeFITS==0:
            spectraFlux = total['flux']
            spectraIvar = total['ivar']
            linlam=np.exp(total['loglam']*np.log(10))          #  ocultar para leer desde archivo
            lambd = linlam/(1+z)
        else:
            spectraFlux = hdulist[0].data #*(1e-19)
            spectraIvar = spectraFlux*0
            if 'CDELT1' in head:
                lambd = np.linspace(head['CRVAL1'], head['CRVAL1']+head['CDELT1']*(len(spectraFlux)-1), len(spectraFlux))
            elif 'W0' in head:
                lambd = np.linspace(head['W0'], head['W0']+head['WPC']*(len(spectraFlux)-1), len(spectraFlux))
            else:
                print "Can't identify FITS format in file: ", str(name)
                break
            lambd = lambd/(1+z)
    else:
        specTable = Table.read(str(name)+'.txt', format='ascii')
        spectraFlux = specTable['Intensity'].data #.tolist()
        spectraIvar = spectraFlux*0
        lambd = specTable['Spectral-axis'].data #.tolist()
        lambd = lambd/(1+z)
    orderU = 10**(int(mt.log(np.mean(spectraFlux), 10))-1)
#    scaleFactor = 10**(-int(mt.log(np.mean(spectraFlux), 10))+1)
#    spectraFlux=spectraFlux*scaleFactor
    
    SpectraIvar.append(spectraIvar)
    TypeFITS.append(typeFITS)
    Lambda.append(lambd)
#    HDUs.append(hdulist)
#    Headers.append(head)   #Datos de la cabecera del archivo .fits
    Spectra.append(spectraFlux)
    print "Objeto: " + name
    spec =np.array(spectraFlux)
    spec2, lambd = r(spec, lambd, paramCont*orderU)
    
    
    limResolucion = mt.ceil(lambd[1]-lambd[0])
    
# ..............    
    
    if lambd[0] > 3720.0 :
        print "The object isn't in expected spectrum region"
        ListaLineas=Table.read('LinesPrincipal.txt', format='ascii')
        LS=np.zeros([13,9])
        
        lineProf = lineProfile(x,spec2, lambd,ListaLineas,Spectra, limResolucion)
        LINEProfile.append(lineProf)
        # Este ciclo calcula remplaza la región del contínuo que está justo en el perfil de línea, por una recta
        # que es mas preciso al determinar la amplitud del modelo gaussiano
        for ina in range(len(lineProf)):
            lInf = lineProf[ina]['inf']
            lSup = lineProf[ina]['sup']
            lDel = lSup-lInf
            if (lInf-3*lDel) >= 0 :
                lInf = lInf-3*lDel
            if (lSup+3*lDel) <= len(lambd) :
                lSup = lSup+3*lDel
            m1, b1 = linModel( lambd[lInf] , spec2[lInf] , lambd[lSup], spec2[lSup])
            lineLambd = lambd[lInf:lSup]
            lineIntCont = lines( lineLambd , m1, b1)
            spec2[lInf:lSup] = lineIntCont
        ModelG1, lineProf, spec2 = finalFit(x, spec2, lambd,ListaLineas,Spectra, spectraIvar,2000, limResolucion)
#        ModelL1 = fit2(x, spec2, lambd,ListaLineas,Spectra,2500)
    else:
        # Este ciclo calcula remplaza la región del contínuo que está justo en el perfil de línea, por una recta
        # que es mas preciso al determinar la amplitud del modelo gaussiano
        ListaLineas=Table.read('LinesAll.txt', format='ascii')
        lineProf = lineProfile(x,spec2, lambd,ListaLineas,Spectra, limResolucion)
        LINEProfile.append(lineProf)
        for ina in range(len(lineProf)):
            lInf = lineProf[ina]['inf']
            lSup = lineProf[ina]['sup']
            lDel = lSup-lInf
            if (lInf-3*lDel) >= 0 :
                lInf = lInf-3*lDel
            if (lSup+3*lDel) <= len(lambd) :
                lSup = lSup+3*lDel
            m1, b1 = linModel( lambd[lInf] , spec2[lInf] , lambd[lSup], spec2[lSup])
            lineLambd = lambd[lInf:lSup]
            lineIntCont = lines( lineLambd , m1, b1)
            spec2[lInf:lSup] = lineIntCont
        LS=np.zeros([len(ListaLineas),9])
        ModelG1, lineProf, spec2 = finalFit(x, spec2, lambd,ListaLineas,Spectra, spectraIvar,2000, limResolucion)
#        ModelL1 = fit2(x, spec2, lambd,ListaLineas,Spectra,2500)
    
    Tabla=Table(LS,names=('Amplitude','Wavelength','Sigma','Lambda Vac','Element','Flux','Flux_Err', 'FWHM', 'eqWidth'),
                dtype=('f8','f8','f8','f8','S8','f8','f8','f8','f8'))
    Tabla['Lambda Vac']=ListaLineas['LAMBDA VAC ANG']
    Tabla['Element']=ListaLineas['SPECTRUM']
    AA=split_list(ModelG1[0].parameters, wanted_parts=len(ListaLineas))
    AAA=split_list(ModelG1[1].parameters, wanted_parts=2)
    SpectraCont.append(spec2)
#    AA=split_list(ModelL1[0].parameters, wanted_parts=len(ListaLineas))
#    AAA=split_list(ModelL1[1].parameters, wanted_parts=2)
    for ii in range(len(ListaLineas)):
        if AA[ii][2]<0:
            Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], mt.fabs(AA[ii][2])])
        else:
            Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=AA[ii]
        if ii==0 or ii==1:
            if AAA[ii][2]<0:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AAA[ii][0], AAA[ii][1], mt.fabs(AAA[ii][2])])
            else:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=AAA[ii]
    Modelos.append(ModelG1)
    #    Modelos.append(ModelL1)
    print Tabla 
    Tablas.append(Tabla)

Objeto: SDSS J003947.13-002934.7
It has not been possible to fit the line:  4341.684
It has not been possible to fit the line:  4862.683
It has not been possible to fit the line:  6529.04
  Amplitude      Wavelength      Sigma      Lambda Vac ... Flux_Err FWHM eqWidth
-------------- ------------- -------------- ---------- ... -------- ---- -------
 3.96996694795 3727.59490796  2.07375842216   3727.092 ...      0.0  0.0     0.0
 1.67668965411 3729.90284345 0.217924034989   3729.875 ...      0.0  0.0     0.0
           0.0      4341.684            0.0   4341.684 ...      0.0  0.0     0.0
 2.26711876954 4364.75555493 0.851010499408   4364.436 ...      0.0  0.0     0.0
           0.0      4862.683            0.0   4862.683 ...      0.0  0.0     0.0
  1.3540461544 4933.07480372 0.772880832301   4932.603 ...      0.0  0.0     0.0
0.866469762038 4955.46084813  8.97011747304   4960.295 ...      0.0  0.0     0.0
  2.4075546355 5006.68370951  2.34535549659    5008.24 ...      0.0  0.0     0.0
  

In [176]:
#   plot of the galaxy with index jj observed with respect to the continuum in zero. Two models for the lines required 

%matplotlib
jj=3
#co = 8
#1343	
#plt.plot(Lambda[jj][1339-co:1346+co],Spectra[jj][1339-co:1346+co], lw=0.5, c = 'k', label='Spectra')

plt.plot(Lambda[jj],Spectra[jj], lw=0.5, c = 'k', label='Spectra')
plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c = 'g', label='Spectra')
#plt.plot(Lambda[jj],spec2[jj], lw=2, c = 'k', label='Spectra')

#plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c='g', label='reducedSpectra')

Using matplotlib backend: Qt4Agg


###     EqWidth ,  FWHM,  FLUX

In [177]:
#   The plot of relative intensity and continuum emission for the galaxy index jj and the wavelenght of all the lines
#   The equivalent width, flux and FWHM for galaxy jj is measured too

#orderU = 10**(int(mt.log(np.mean(spectraFlux), 10))-1)

#spec =np.array(spectraFlux)
#spec2, lambd = r(spec, lambd, 0.004*orderU) # este parámetro de 0.004 puede ser igual para todas las galaxias
                                      #  pero seria bueno iterar sobre este valor para tener mejores continuos
                                      #  o tal vez tomar tres o maś regiones del espectro observado y calcular spec
                                    #  para diferentes parámetros de 0.1 a 0.001
        

#%matplotlib
for jj in range(len(resultListDT['ObjID'])):
    spe = []
    speCont = []
    for xind in range(len(Spectra[jj])):
        spe.append((Spectra[jj][xind])/(SpectraCont[jj][xind]))
        speCont.append((SpectraCont[jj][xind])/(SpectraCont[jj][xind]))
    #for i in range(len(Tablas[jj])):
    #    plt.axvline(x=Tablas[jj]['Lambda Vac'][i],c='dimgrey',lw=0.8,ls='--')
    #    plt.text(Tablas[jj]['Lambda Vac'][i] + 0.5, Tablas[jj]['Amplitude'][i] * 1.2,Tablas[jj]['Element'][i])

    #plt.plot(lambd, spe, c='k', lw=0.5, label='normalized data')
    #plt.plot(lambd, speCont, lw=2, label='normalized continuum')

    lFWHM = []
    eqW = []    ##  lista de valores 
    #  una vez normalizado el espectro observado a la medida del contínuo se integra por el método de Simpson
    for iLineProf in range(len(LINEProfile[jj])):
        f = FWHM(Lambda[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], Spectra[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]])
        if len(f) == 0:
            lFWHM.append(0)
        else:
            lFWHM.append(f[0])
        sInteg = simps(y=spe[LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], x=Lambda[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], even='avg')
        eqW.append(sInteg)
    #   Flux determination for each model in galaxy list
    #   assignment of the FWHM and equivalent width in the Table for the first galaxy
    for i in range(len(Tablas[jj])):
        if Tablas[jj][i]['Amplitude']<=0:
            Tablas[jj]['Flux'][i],Tablas[jj]['Flux_Err'][i]= 0,0
        else:
            Tablas[jj]['Flux'][i],Tablas[jj]['Flux_Err'][i]=quad(gaussian,Tablas[jj][i]['Wavelength']-3*Tablas[jj][i]['Sigma'],Tablas[jj][i]['Wavelength']+3*Tablas[jj][i]['Sigma'],args=(Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']))
            Tablas[jj]['FWHM'][i],Tablas[jj]['eqWidth'][i] = lFWHM[i], eqW[i]
    Tablas[jj].write('Tabla'+str(resultListDT['ObjID'][jj])+'.tbl',format='ascii')


###        Electron density and temperature and plot

In [178]:
#   it obtains the electron density and temperature by the PyNeb cross iterative method for all couple of given species.

fig = plt.figure()
fig.suptitle('')
ax = fig.add_subplot(111)
ax.set_ylabel('Density $n_e (cm^{-3})$')
ax.set_xlabel('Temperature $T_e (K)$')

for jj in range(len(resultListDT)):
    k=0
    if Lambda[jj][0] > 3720.0 :
        k=2
    try:
        a=diags.getCrossTemDen('[OIII] 4363/5007+', '[SII] 6731/6716', Tablas[jj]['Flux'][3-k]/(Tablas[jj]['Flux'][6-k]+Tablas[jj]['Flux'][7-k]), Tablas[jj]['Flux'][13-k]/Tablas[jj]['Flux'][14-k])
        b=diags.getCrossTemDen('[NII] 5755/6584+', '[OII] 3726/3729', Tablas[jj]['Flux'][8-k]/(Tablas[jj]['Flux'][12-k]+Tablas[jj]['Flux'][10-k]), Tablas[jj]['Flux'][0-k]/Tablas[jj]['Flux'][1-k])
        c=diags.getCrossTemDen('[NII] 5755/6584+', '[SII] 6731/6716', Tablas[jj]['Flux'][8-k]/(Tablas[jj]['Flux'][12-k]+Tablas[jj]['Flux'][10-k]), Tablas[jj]['Flux'][13-k]/Tablas[jj]['Flux'][14-k])
        d=diags.getCrossTemDen('[OIII] 4363/5007+', '[OII] 3726/3729', Tablas[jj]['Flux'][3-k]/(Tablas[jj]['Flux'][6-k]+Tablas[jj]['Flux'][7-k]), Tablas[jj]['Flux'][0-k]/Tablas[jj]['Flux'][1-k])
    except ValueError:
        continue

    for i in IntrinsicHB:
        rc.setCorr((Tablas[jj]['Flux'][11-k]/Tablas[jj]['Flux'][4-k])/i, Tablas[jj]['Lambda Vac'][11-k], Tablas[jj]['Lambda Vac'][4-k])
        try:
            a2=diags.getCrossTemDen('[OIII] 4363/5007+', '[SII] 6731/6716',Tablas[jj]['Flux'][3-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][3-k])/(Tablas[jj]['Flux'][6-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][6-k])+ Tablas[jj]['Flux'][7-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k]) ), Tablas[jj]['Flux'][13-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][13-k])/Tablas[jj]['Flux'][14-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][14-k]))
            b2=diags.getCrossTemDen('[NII] 5755/6584+', '[OII] 3726/3729', Tablas[jj]['Flux'][8-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][8-k])/(Tablas[jj]['Flux'][12-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][12-k])+Tablas[jj]['Flux'][10-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k])), Tablas[jj]['Flux'][0-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][0-k])/Tablas[jj]['Flux'][1-k])*rc.getCorrHb(Tablas[jj]['Lambda Vac'][1-k])
            c2=diags.getCrossTemDen('[NII] 5755/6584+', '[SII] 6731/6716', Tablas[jj]['Flux'][8-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][8-k])/(Tablas[jj]['Flux'][12-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][12-k])+Tablas[jj]['Flux'][10-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k])),Tablas[jj]['Flux'][13-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][13-k])/Tablas[jj]['Flux'][14-k] * rc.getCorrHb(Tablas[jj]['Lambda Vac'][14-k]))
            d2=diags.getCrossTemDen('[OIII] 4363/5007+', '[OII] 3726/3729',Tablas[jj]['Flux'][3-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][3-k])/(Tablas[jj]['Flux'][6-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][6-k])+ Tablas[jj]['Flux'][7-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][7-k]) ), Tablas[jj]['Flux'][0-k]*rc.getCorrHb(Tablas[jj]['Lambda Vac'][0-k])/Tablas[jj]['Flux'][1-k])*rc.getCorrHb(Tablas[jj]['Lambda Vac'][1-k])
        except ValueError:
            continue
        if b2 != () or d2 != ():
            plt.scatter(b2[0], b2[1],c='c')
            plt.scatter(d2[0], d2[1],c='c')
        if i == 2.86:
            resultListDT['Temp_1 [OIII]'][jj]=a2[0]
            resultListDT['Den_1 [SII]'][jj]=a2[1]
            resultListDT['Temp_3 [NII]'][jj]=c2[0]
            resultListDT['Den_3 [SII]'][jj]=c2[1]
            if b2 != () or d2 != ():
                resultListDT['Temp_2 [NII]'][jj]=b2[0]
                resultListDT['Den_2 [OII]'][jj]=b2[1] 
                resultListDT['Temp_4 [OIII]'][jj]=d2[0]
                resultListDT['Den_4 [OII]'][jj]=d2[1]
            if lambd[0] > 3720.0 :
                resultListDT['Temp_2 [NII]'][jj]=0.0
                resultListDT['Den_2 [OII]'][jj]=0.0
                resultListDT['Temp_4 [OIII]'][jj]=0.0
                resultListDT['Den_4 [OII]'][jj]=0.0
    resultListDT.write('TemDenGalaxies'+str(resultListDT['ObjID'][jj])+'.tbl',format='ascii')
print Tablas
resultListDT


[<Table masked=False length=15>
  Amplitude      Wavelength      Sigma      ...      FWHM        eqWidth   
   float64        float64       float64     ...    float64       float64   
-------------- ------------- -------------- ... ------------- -------------
 3.96996694795 3727.59490796  2.07375842216 ... 5.14794921875 11.0666942596
 1.67668965411 3729.90284345 0.217924034989 ... 5.14794921875 11.0666942596
           0.0      4341.684            0.0 ...           0.0           0.0
 2.26711876954 4364.75555493 0.851010499408 ... 2.01025390625   5.583383366
           0.0      4862.683            0.0 ...           0.0           0.0
  1.3540461544 4933.07480372 0.772880832301 ... 3.40576171875 4.76902866364
0.866469762038 4955.46084813  8.97011747304 ...  1.1435546875 2.42871212959
  2.4075546355 5006.68370951  2.34535549659 ...   4.611328125 7.97471570969
           0.0       5756.24            0.0 ...           0.0           0.0
           0.0       6529.04            0.0 ...         

/usr/lib/python2.7/site-packages/IPython/kernel/__main__.py:22: RuntimeWarning: divide by zero encountered in double_scalars


ObjID,ra,dec,z,isFits,Temp_1 [OIII],Den_1 [SII],Temp_2 [NII],Den_2 [OII],Temp_3 [NII],Den_3 [SII],Temp_4 [OIII],Den_4 [OII]
,,,,,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3
string192,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64
SDSS J003947.13-002934.7,9.94641685486,-0.49302777648,0.222564,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2MASX J00400069-0051457,10.0030412674,-0.862777769566,0.111082,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SDSS J004119.73+002632.3,10.3322086334,0.44230556488,0.108194,1,nan,nan,nan,nan,nan,nan,nan,nan
2MASX J00412528+1343345,10.3554162979,13.7261943817,0.033304,1,nan,nan,nan,nan,nan,nan,nan,nan


###                     Plots

In [170]:
#   plot of the galaxy with index jj observed with respect to the continuum in zero. Two models for the lines required 

%matplotlib
jj=0
if isFits:
    if TypeFITS == 0:
        print HDUs[jj][0].header['FLAVOR']
        print HDUs[jj][0].header['QUALITY']
        print HDUs[jj][0].header['FBADPIX'] ## %%Malos Pixeles
        print HDUs[jj][2].data['SN_MEDIAN_ALL']
        print HDUs[jj][2].data['SN_MEDIAN']

lambd=Lambda[jj]

plt.figure(figsize=(16, 9))
plt.title('Spectrum Galaxy '+ resultListDT['ObjID'][jj])
plt.xlabel(r'Wavelength (Angstroms $\AA$) ')
if TypeFITS == 0:
    plt.ylabel(r'$f_{\lambda}$ ('+str(head['BUNIT'])+')')
else:
    plt.ylabel(r'$f_{\lambda}$')

plt.errorbar(Lambda[jj],Spectra[jj]-SpectraCont[jj],SpectraIvar[jj],  c='k',lw=0.5,label='Data')   #Flujo en negro
#plt.errorbar(Lambda[jj],Spectra[jj]-SpectraCont[jj]-Modelos[jj][0](Lambda[jj]),SpectraIvar[jj],  c='k',lw=2,label='Data')   #Flujo en negro
#plt.errorbar(lambd,Spectra[jj] ,SpectraIvar[jj],  c='k',lw=2,label='Data')   #Flujo en negro
#plt.plot(lambd, spec2, lw=2, label='reducedSpectra')
plt.plot(Lambda[jj], Modelos[jj][1](Lambda[jj]), c='midnightblue', lw=2, label='Model')
plt.plot(Lambda[jj], Modelos[jj][0](Lambda[jj]), c='yellow', lw=1, label='Model')
#plt.plot(lambd, spec2, lw=2, label='reducedSpectra')

plt.legend()

x=np.linspace(Lambda[jj][0],Lambda[jj][-1],len(Lambda[jj]))
for i in range(len(Tablas[jj])):
    plt.axvline(x=Tablas[jj]['Lambda Vac'][i],c='dimgrey',lw=1.2,ls='--')
    plt.text(Tablas[jj]['Lambda Vac'][i] + 0.5, Tablas[jj]['Amplitude'][i] * 1.2,Tablas[jj]['Element'][i])
    if Tablas[jj]['Element'][i] == 'H I':
        plt.plot(x,gaussian(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']), c='DeepPink')
##        plt.plot(x,lorentzFunc(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Sigma'], Tablas[jj][i]['Wavelength']), c='DeepPink')
    else:
        if Tablas[jj][i]['Sigma']!=0:
            plt.plot(x,gaussian(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']) , c='Lime')
##            plt.plot(x,lorentzFunc(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Sigma'],Tablas[jj][i]['Wavelength']) , c='Lime')


Using matplotlib backend: Qt4Agg


In [14]:
%matplotlib
kaksj = Modelos[jj][1](lambd)
lalskal = Modelos[jj][0](lambd)
specModel = spec+lalskal
plt.errorbar(lambd,Spectra[jj] ,SpectraIvar[jj],  c='k',lw=2,label='Data')   #Flujo en negro
plt.plot(lambd, specModel, lw=2, label='reducedSpectra')

Using matplotlib backend: Qt4Agg


In [14]:
%matplotlib

plt.figure(figsize=(16, 9))
for i in range(10):
    B=[]
    for j in range(len(Spectra[jj])):
        bb=SpectraIvar[jj][j]*random.uniform(-3,3)
        B.append(bb)
    plt.plot(lambd,(Spectra[jj] +B),c='orange')
plt.legend()

Using matplotlib backend: Qt4Agg


/usr/local/lib/python2.7/dist-packages/matplotlib/axes/_axes.py:475: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [117]:
resultList = Table.read("resultList.tbl", format = 'ascii')
OIII5007_WT = 5008.24
Hb_WT = 4862.683
NII6583_WT = 6585.28
Ha_WT = 6564.61
SII6717_WT = 6718.29
SII6731_WT = 6732.67

%matplotlib

#fig = plt.figure()
#ax = fig.add_subplot(2,1,1)
#ax.set_yscale('log')

plt.yscale('log')
plt.xscale('log')
cooo = -1
for objID in resultList:
    nameTable = "Tabla"+str(objID[0])+".tbl"
    tableObjID = Table.read(nameTable, format = 'ascii')
    OIII5007 = tableObjID["Flux"][7]
    Hb = tableObjID["Flux"][4]
    NII6583 = tableObjID["Flux"][12]
    Ha = tableObjID["Flux"][11] 
    SII6717 = tableObjID["Flux"][13]
    SII6731 = tableObjID["Flux"][14]               # +++++++++++++++
    OIII5007_W = tableObjID["Wavelength"][7]
    Hb_W = tableObjID["Wavelength"][4]
    NII6583_W = tableObjID["Wavelength"][12]
    Ha_W = tableObjID["Wavelength"][11]
    SII6717_W = tableObjID["Wavelength"][13]
    SII6731_W = tableObjID["Wavelength"][14]
    if OIII5007>0.0 and Hb>0.0 and NII6583>0.0 and Ha>0.0 and mt.fabs(OIII5007_WT-OIII5007_W)<1.5 and mt.fabs(Hb_WT-Hb_W)<1.5 and mt.fabs(NII6583_WT-NII6583_W)<1.5 and mt.fabs(Ha_WT-Ha_W)<1.5:
        #    print NII6583/Ha, OIII5007/Hb, nameTable
        cooo = cooo + 1
        plt.scatter([NII6583/Ha], [OIII5007/Hb], c='k' )
    
#    if OIII5007>0.0 and Hb>0.0 and SII6717>0.0 and SII6731>0.0 and Ha>0.0 and mt.fabs(OIII5007_WT-OIII5007_W)<1.5 and mt.fabs(Hb_WT-Hb_W)<1.5 and mt.fabs(SII6731_WT-SII6731_W)<1.5 and mt.fabs(SII6717_WT-SII6717_W)<1.5 and mt.fabs(Ha_WT-Ha_W)<1.5:
#        #    print NII6583/Ha, OIII5007/Hb, nameTable
#        cooo = cooo + 1
#        plt.scatter([(SII6717+SII6731)/Ha], [OIII5007/Hb], c='k' )
    
    #print "OIII: ",mt.fabs(OIII5007_WT-OIII5007_W), "Hbeta: ",mt.fabs(Hb_WT-Hb_W), "NII: ",mt.fabs(NII6583_WT-NII6583_W), "Halpha: ",mt.fabs(Ha_WT-Ha_W)  #, nameTable

Using matplotlib backend: Qt4Agg


In [115]:
cooo

32

In [103]:
tableObjID

Amplitude,Wavelength,Sigma,Lambda Vac,Element,Flux,Flux_Err,FWHM,eqWidth
float64,float64,float64,float64,string56,float64,float64,float64,float64
0.0,3727.092,0.0,3727.092,[O II],0.0,0.0,0.0,0.0
10.9654041435,3731.88461377,2.78840877294,3729.875,[O II],76.4358193343,8.48608062618e-13,0.0,6.33787602988
35.0743561758,4346.14755701,6.60866080193,4341.684,H I,579.454061974,6.433232419e-12,0.0,3.88139414744
11.0222530334,4360.57138222,4.22433059715,4364.436,[O III],116.397624632,1.29227322745e-12,0.0,4.40042352676
124.443802963,4866.69100447,10.843008098,4862.683,H I,3373.17520008,3.74497676793e-11,14.5698242188,58.6926002502
17.3335028207,4930.13960784,6.19581803732,4932.603,[O III],268.473133759,2.9806505571e-12,0.0,3.92683624208
17.0652004245,4958.72656613,3.69791581698,4960.295,[O III],157.755407761,1.75143685904e-12,0.0,6.34411865537
90.3054177862,5007.83987015,8.43299817755,5008.24,[O III],1903.75762006,2.11359553991e-11,0.0,19.689653188
3.9936753097,5760.40097659,6.00545588554,5756.24,[N II],59.9562658294,6.65648268385e-13,0.0,2.75366020203


In [27]:
spec[len(spec)-1]

6.8772826

In [104]:
len(resultList)

64